In [ ]:
from importlib import reload
import generate_data_functions
reload(generate_data_functions)
from generate_data_functions import *

from importlib import reload
import grouped_iForest_functions
reload(grouped_iForest_functions)
from grouped_iForest_functions import *

from sklearn.ensemble import IsolationForest
from statistics import mean

def summarization(df):        
    # Summarize the data by grouping by IP address and counting port and service occurrences
    df_sum = summarize_features(df)
    print(df.columns)
    '''print(df_sum.columns)
    print('df_sum',df_sum)'''

    '''S = set(df_sum[df_sum['anomaly'] == 1]['ip_id'].unique())
    df_filtered = df_sum[df_sum['ip_id'].isin(S)]
    #print(df_filtered.drop(columns = ['ip_id']))'''

    df_sum = df_sum.rename(str,axis="columns")
    all_columns = df_sum.columns.tolist()
    column_to_exclude = ['ipv4', 'anomaly']
    features = [col for col in all_columns if col not in column_to_exclude]

    regForest_sum = IsolationForest(random_state=0)
    regForest_sum.fit(df_sum[features])
    regForest_sum_labels = regForest_sum.predict(df_sum[features])
    regForest_sum_scores = regForest_sum.decision_function(df_sum[features])
    df_sum['anomaly_score'] = regForest_sum_scores

    sum_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for index, row in df_sum.iterrows():
        ipv4 = row['ipv4']
        anomaly_score = row['anomaly_score']
        sum_anomaly_dict[ipv4] = anomaly_score
    return sum_anomaly_dict

def pairs_anomaly(df):
    # Run Data Version (IP-port pairs) 2
    # No services in this data version
    print('2 PAIRS')

    df_pairs = pairs(df)
    #print("df_pairs", df_pairs)
    # Display the feature matrix
    all_columns = df_pairs.columns.tolist()

    #Specify the column to exclude
    column_to_exclude = ['ipv4', 'anomaly']

    # Create a new list excluding the specified column
    features = [col for col in all_columns if col not in column_to_exclude]
    #print('features:', features)
    regForest_pairs = IsolationForest(random_state=0)
    regForest_pairs.fit(df_pairs[features])
    regForest_pairs_labels = regForest_pairs.predict(df_pairs[features])
    regForest_pairs_scores = regForest_pairs.decision_function(df_pairs[features])  
    df_pairs['anomaly_score'] = regForest_pairs_scores

    # Group by 'ipv4' and sum anomaly scores
    anomalies_sum = df_pairs.groupby('ipv4')['anomaly_score'].sum()
    anomalies_count = df_pairs['ipv4'].value_counts()

    anomalies_avg = anomalies_sum / anomalies_count

    # Sort anomalies by their average score
    sorted_anomalies_avg = anomalies_avg.sort_values()

    # Create the dictionary to store the results
    pairs_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in sorted_anomalies_avg.items():
        pairs_anomaly_dict[ipv4] = anomaly_score
    return pairs_anomaly_dict

def cust_iforest_anomaly(df):
    # prepocesses the data by flattening it and separating IPs into octets
    features_flat = preprocess_customiForest(df)
    custForest_flat = CustomIsolationForest(ipv4_index=0, n_estimators=2, random_state=0, df=df)
    custForest_flat.fit(features_flat)

#*********** HERE IS WHERE YOU CAN CHANGE THE MIN_SCORE TO TRUE OR FALSE *************
    #if min_score = False, iForest will determine the anoamly by taking the average score rather than the lowest (most anaomalous) score
    custForest_flat_labels = custForest_flat.predict(features_flat, min_score=True)
    custForest_flat_scores = custForest_flat.decision_function(features_flat)
    return custForest_flat_scores

def flattened(df):
    # Run Data Version (original - flattened) 1
    features_flat = preprocess_customiForest(df)
    #print('features_flat:', features_flat)
    regForest_flat = IsolationForest(random_state=0)
    regForest_flat.fit(features_flat)
    regForest_flat_labels = regForest_flat.predict(features_flat)
    regForest_flat_scores = regForest_flat.decision_function(features_flat)    
    features_flat['anomaly_score'] = regForest_flat_scores

    # Group by 'ipv4' and sum anomaly scores
    features_flat['ipv4'] = features_flat[['octet1', 'octet2', 'octet3', 'octet4']].astype(str).agg('.'.join, axis=1)
    anomalies_sum = features_flat.groupby('ipv4')['anomaly_score'].sum().reset_index()
    anomalies_count = features_flat['ipv4'].value_counts().reset_index(name='count')
    merged = pd.merge(anomalies_sum, anomalies_count, on='ipv4')
    anomalies_avg = merged['anomaly_score'] / merged['count']
    
    reg_anomaly_dict = {}

    # Populate the dictionary with IP addresses and their anomaly scores
    for ipv4, anomaly_score in anomalies_avg.items():
        reg_anomaly_dict[ipv4] = anomaly_score
    return reg_anomaly_dict

def calculate_f2_score(true_anomalous_ips, predicted_anomalous):
    """
    Calculates the F2-Score based on true anomalous IPs and predicted anomalous IPs.
    
    Parameters:
    - true_anomalous_ips (list): List of true anomalous IP addresses.
    - predicted_anomalous (list of tuples): List of (IP, score) for IPs flagged as anomalous.
    
    Returns:
    - float: The F2-Score for the model.
    """
    
    # Convert inputs to sets of IPs for comparison
    true_anomalous_set = set(true_anomalous_ips)
    predicted_anomalous_set = set(ip for ip, score in predicted_anomalous)
    
    # Calculate true positives, false positives, and false negatives
    tp = len(true_anomalous_set & predicted_anomalous_set)
    fp = len(predicted_anomalous_set - true_anomalous_set)
    fn = len(true_anomalous_set - predicted_anomalous_set)
    
    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Calculate F2-Score with beta = 2
    f2_score = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall) if (precision + recall) > 0 else 0
    
    return f2_score

def calculate_metrics(true_anomalous_ips, predicted_anomalous):
    """
    Calculates Precision, Recall, and F2-Score based on true and predicted anomalous IPs.
    
    Parameters:
    - true_anomalous_ips (list): List of true anomalous IP addresses
    - predicted_anomalous (list of tuples): List of (IP, score) for IPs flagged as anomalous
    
    Returns:
    - tuple: (precision, recall, f2_score)
    """
    true_anomalous_set = set(true_anomalous_ips)
    predicted_anomalous_set = set(ip for ip, score in predicted_anomalous)
    
    tp = len(true_anomalous_set & predicted_anomalous_set)
    fp = len(predicted_anomalous_set - true_anomalous_set)
    fn = len(true_anomalous_set - predicted_anomalous_set)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    f2_score = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f2_score

typical_service_port_map = {
    'FTP': 21,
    'SMTP': 25,
    'DNS': 53,
    'HTTP': 80
    }
    #'POP3': 110,
    #'IMAP': 143,
    #'MYSQL': 3306,
    #'SSH': 22
#}

# region parameters
num_repetitions = 5
seeds = range(num_repetitions)
num_ips=5
num_scans_per_ip=20
anomaly_rate=0.2
anomalous_scan_prob = 0.1
# endregion

all_exp = [] # List to store all experiments
# Lists to store metrics for each method
metrics_custom = {'precision': [], 'recall': [], 'f2': []}
metrics_reg = {'precision': [], 'recall': [], 'f2': []}
metrics_pairs = {'precision': [], 'recall': [], 'f2': []}
metrics_sum = {'precision': [], 'recall': [], 'f2': []}

# Loop over seeds
for seed in seeds:
    # Set the random seed for reproducibility
    np.random.seed(seed)
    
    # region generate the data
    df = generate_scan_data_with_variance(num_ips, num_scans_per_ip, typical_service_port_map, anomaly_rate, anomalous_scan_prob, seed)

    S = set(df[df['anomaly'] == 1]['ip_id'].unique())
    df_filtered = df[df['ip_id'].isin(S) & df['anomaly'] == 1]
    print(df_filtered.drop(columns = ['ip_id']))

    # Add some hand written anomalies to the data
    '''anomalous_scans = [
    ['198.51.100.1', 0, 0, [80, 22, 21], ['HTTP', 'SSH', 'FTP'], 0],
    ['198.51.100.1', 0, 1, [143, 3306], ['IMAP', 'MYSQL'], 0],
    ['198.51.100.1', 0, 2, [80, 143], ['HTTP', 'IMAP'], 0],
    ['198.51.100.1', 0, 3, [22, 110], ['SSH', 'POP3'], 0],
    ['198.51.100.1', 0, 4, [53, 3306], ['DNS', 'MYSQL'], 0],
    ['198.51.100.1', 0, 5, [80, 3306], ['HTTP', 'MYSQL'], 1], # Anomaly: MYSQL on port 80
    ['198.51.100.1', 0, 6, [21, 143], ['FTP', 'IMAP'], 1],  # Anomaly: IMAP on port 21
    ['198.51.100.1', 0, 7, [22, 53], ['SSH', 'DNS'], 0],
    ['198.51.100.1', 0, 8, [110, 3306], ['POP3', 'MYSQL'], 0],
    ['198.51.100.1', 0, 9, [21, 143], ['FTP', 'IMAP'], 0]
]
    
    anomalous_df = pd.DataFrame(anomalous_scans, columns=['ipv4', 'ip_id', 'scan_id', 'port', 'service', 'anomaly'])

    df = pd.concat([df, anomalous_df], ignore_index=True)'''
    
    anomalous_ips = df[df['anomaly'] == 1]['ipv4'].unique()
    print('anomalous_ips:', anomalous_ips)
    # endregion 

    # region compute anomalies
    custForest_flat_scores = cust_iforest_anomaly(df)

    reg_anomaly_dict = flattened(df)

    pairs_anomaly_dict = pairs_anomaly(df)

    sum_anomaly_dict = summarization(df)
    
    # Anomalous IPs from the Custom Isolation Forest (Flattening)
    custForest_anomalous_ips = [(ip, score) for ip, score in custForest_flat_scores.items() if score < 0]
     # Anomalous IPs from the Standard Isolation Forest (Flattening)
    regForest_anomalous_ips = [(ip, score) for ip, score in reg_anomaly_dict.items() if score < 0]
    # Standard Isolation Forest (IP-Port Pairs): (IP, score) pairs for anomalous IPs
    pairs_anomalous_ips = [(ip, score) for ip, score in pairs_anomaly_dict.items() if score < 0]
    # Standard Isolation Forest (Summarization): (IP, score) pairs for anomalous IPs
    sum_anomalous_ips = [(ip, score) for ip, score in sum_anomaly_dict.items() if score < 0]
    # endregion

    # region F2 scores
    # Calculate metrics for each method
    custom_prec, custom_rec, custom_f2 = calculate_metrics(anomalous_ips, custForest_anomalous_ips)
    reg_prec, reg_rec, reg_f2 = calculate_metrics(anomalous_ips, regForest_anomalous_ips)
    pairs_prec, pairs_rec, pairs_f2 = calculate_metrics(anomalous_ips, pairs_anomalous_ips)
    sum_prec, sum_rec, sum_f2 = calculate_metrics(anomalous_ips, sum_anomalous_ips)
    
    # Store metrics
    metrics_custom['precision'].append(custom_prec)
    metrics_custom['recall'].append(custom_rec)
    metrics_custom['f2'].append(custom_f2)

    metrics_reg['precision'].append(reg_prec)
    metrics_reg['recall'].append(reg_rec)
    metrics_reg['f2'].append(reg_f2)
    
    metrics_pairs['precision'].append(pairs_prec)
    metrics_pairs['recall'].append(pairs_rec)
    metrics_pairs['f2'].append(pairs_f2)
    
    metrics_sum['precision'].append(sum_prec)
    metrics_sum['recall'].append(sum_rec)
    metrics_sum['f2'].append(sum_f2)    
    # endregion

    # Store all information in the experiment dictionary
    exp = {
        'seed': seed,
        'num_ips': num_ips,
        'num_scans_per_ip': num_scans_per_ip,
        "Custom iForest Flattening scores": custForest_flat_scores,
        "iForest Pairs scores": pairs_anomaly_dict,
        "iForest Summarization scores": sum_anomaly_dict,
        "Actual Anomalous IPs": anomalous_ips,
        "Custom iForest Anomalous IPs": custForest_anomalous_ips,
        "iForest Pairs Anomalous IPs": pairs_anomalous_ips,
        "iForest Summarization Anomalous IPs": sum_anomalous_ips,
        "Custom iForest F2-Score": custom_f2,
        "iForest Pairs F2-Score": custom_f2,
        "iForest Summarization F2-Score": custom_f2
    }
    
    # Add the experiment dictionary to the list of all experiments
    all_exp.append(exp)

#region Print all experiments for verification
'''
for experiment in all_exp:
    for key, value in experiment.items():
        print(f"{key}: {value}")
    print("-" * 20)  # Separator between experiments
    '''
#endregion

# Calculate averages
avg_metrics = {
    'Custom iForest': {
        'Precision': mean(metrics_custom['precision']),
        'Recall': mean(metrics_custom['recall']),
        'F2-Score': mean(metrics_custom['f2'])
    },
    'iForest Flattened': {
        'Precision': mean(metrics_reg['precision']),
        'Recall': mean(metrics_reg['recall']),
        'F2-Score': mean(metrics_reg['f2'])
    },
    'iForest Pairs': {
        'Precision': mean(metrics_pairs['precision']),
        'Recall': mean(metrics_pairs['recall']),
        'F2-Score': mean(metrics_pairs['f2'])
    },
    'iForest Summarization': {
        'Precision': mean(metrics_sum['precision']),
        'Recall': mean(metrics_sum['recall']),
        'F2-Score': mean(metrics_sum['f2'])
    }
}

# Create and display results table
results_df = pd.DataFrame(avg_metrics).round(3)
print("\nAverage Performance Metrics:")
print("-" * 63)
print(results_df)
print("-" * 63)

Empty DataFrame
Columns: [ipv4, scan_id, port, service, anomaly]
Index: []
anomalous_ips: []
2 PAIRS
Index(['ipv4', 'ip_id', 'scan_id', 'port', 'service', 'anomaly'], dtype='object')
Anomalous ip: 107.48.249.14
Anomalous ip: 162.15.11.13
             ipv4  scan_id              port                 service  anomaly
58  107.48.249.14       58  [53, 80, 21, 25]  [FTP, SMTP, DNS, HTTP]        1
59  107.48.249.14       59  [53, 80, 21, 25]  [FTP, SMTP, DNS, HTTP]        1
98   162.15.11.13       98  [25, 80, 53, 21]  [FTP, SMTP, DNS, HTTP]        1
99   162.15.11.13       99  [25, 80, 53, 21]  [FTP, SMTP, DNS, HTTP]        1
anomalous_ips: ['107.48.249.14' '162.15.11.13']
2 PAIRS
Index(['ipv4', 'ip_id', 'scan_id', 'port', 'service', 'anomaly'], dtype='object')
Anomalous ip: 86.157.128.108
              ipv4  scan_id              port                 service  anomaly
38  86.157.128.108       38  [53, 80, 21, 25]  [FTP, SMTP, DNS, HTTP]        1
39  86.157.128.108       39  [53, 80, 21, 25]  

In [52]:
random.randint(3, 4)

4